# Guide Recommendation Pipeline
The guide recommendation system aims to provide the tourists with suggestions about some potentially interesting guides for them, according to the users' preferences and requests.

Recommendation systems are decision-support systems which can identify and learn patterns among sources of information. They are able to achieve great performance when a good amount of high-quality data is available.

The designed system relies upon datasets of:
- Ratings given by tourists to specific guides;
- Set of attributes about the guides;
- Set of attributes about the tourists.

## Define working environment

In [1]:
# Import libraries

import numpy as np
import pandas as pd
import scipy.sparse as sps
import ast

from tqdm import tqdm

## Load and preprocess data

In this part we are loading the previously generated dataframes:
- tourists.csv with the id, languages spoken, and keywords of the tourists.
- guides.csv with various attributes of the guides.
- guides_ratings.csv which contains the tourist-guide interactions.


In [2]:
# Load dataframe for tourist attributes

tourist_file = open('Data/tourists_500.csv')

tourist_df = pd.read_csv(
    filepath_or_buffer = tourist_file,
    sep = ';',
    header = 0
)

tourist_df.rename(columns={tourist_df.columns[0]: 'id'}, inplace=True)

tourist_df

,id,languages,keywords
0,0,['spanish'],"['rafting', 'countryside', 'literature']"
1,1,['chinese'],"['art', 'wine', 'literature', 'sport']"
2,2,['italian'],"['sport', 'rafting', 'cinema', 'museums', 'mus..."
3,3,['spanish'],['beer']
4,4,['french'],"['tracking', 'food', 'art', 'countryside', 'ra..."
...,...,...,...
495,495,['french'],"['music', 'museums', 'literature', 'cinema', '..."
496,496,['bulgarian'],"['countryside', 'food', 'wine', 'music', 'hist..."
497,497,['french'],"['music', 'beer', 'rafting', 'museums', 'track..."
498,498,['italian'],['sport']


In [3]:
# Load dataframe for guide attributes

guide_file = open('Data/guides_50.csv')

guide_df = pd.read_csv(
    filepath_or_buffer = guide_file,
    sep = ';',
    header = 0,

    # Process the columns containing lists: load the content of those columns as list
    converters = {'languages_spoken':ast.literal_eval, 'keywords':ast.literal_eval}
)

guide_df.rename(columns={guide_df.columns[0]: 'id'}, inplace=True)

guide_df

,id,gender,name,birth_date,now_available,languages_spoken,price,education,biography,keywords,current_location,experience
0,0,male,Henry Robles,1996-10-16,True,[english],25,high-school,Community development worker,[museums],"{'lat': 40.342693584880706, 'lon': 18.16438078...",9
1,1,female,Raven Burke,2004-05-14,True,"[italian, dutch]",36,middle-school,Quantity surveyor,"[cinema, rafting, history, wine]","{'lat': 40.3367551413547, 'lon': 18.1569120995...",1
2,2,female,Nancy Keith,1984-08-04,True,"[chinese, french, english]",34,phd,Secretary/administrator,"[food, archeology, art]","{'lat': 40.362447049660204, 'lon': 18.14225129...",18
3,3,male,Michael Mitchell,1981-08-22,True,[bulgarian],46,bachelor,"Lighting technician, broadcasting/film/video","[countryside, rafting, art]","{'lat': 40.36584086550253, 'lon': 18.183002910...",18
4,4,male,John Bailey,1971-10-09,True,"[deutsche, french]",30,master,Estate manager/land agent,"[countryside, tracking, beer]","{'lat': 40.354724889812935, 'lon': 18.20308322...",22
5,5,male,Justin Allen,1987-10-04,True,"[deutsche, dutch, bulgarian]",27,middle-school,Microbiologist,[],"{'lat': 40.35854623058413, 'lon': 18.183459879...",4
6,6,male,Jose Russell,1995-01-09,True,"[english, chinese, french]",34,middle-school,"Pharmacist, community","[rafting, art, sport]","{'lat': 40.35774331848761, 'lon': 18.163273185...",1
7,7,male,Jack Smith,1981-07-28,True,"[spanish, dutch, french, italian]",33,bachelor,Theatre manager,"[museums, art, countryside, wine]","{'lat': 40.365193043282794, 'lon': 18.18548169...",5
8,8,male,Victor Gonzalez,2002-09-04,True,"[bulgarian, english, chinese]",31,middle-school,Geochemist,"[literature, cinema, music]","{'lat': 40.35265552781134, 'lon': 18.151723396...",4
9,9,male,John Brown,1990-02-18,True,"[chinese, bulgarian, dutch, english]",37,phd,"Development worker, international aid",[museums],"{'lat': 40.33774657696426, 'lon': 18.182064664...",15


In [4]:
# Load dataframe for ratings

rating_file = open('Data/guides_ratings_500_50.csv')

rating_df = pd.read_csv(
    filepath_or_buffer = rating_file,
    sep = ';',
    header = 0
)

rating_df

,0,1,2
0,0,36,4.0
1,0,41,3.0
2,1,45,5.0
3,2,48,4.0
4,2,47,4.0
...,...,...,...
737,496,48,4.0
738,497,7,5.0
739,498,48,4.0
740,498,23,5.0


In [5]:
# Rename the columns of the rating dataframe
rating_df.rename(columns={rating_df.columns[0]: 'tourist_id',
                          rating_df.columns[1]: 'guide_id',
                          rating_df.columns[2]: 'rating',
                         }, inplace=True)

# Remove the duplicated interactions, if any
rating_df.drop_duplicates(subset=['tourist_id','guide_id'],inplace=True)

rating_df

,tourist_id,guide_id,rating
0,0,36,4.0
1,0,41,3.0
2,1,45,5.0
3,2,48,4.0
4,2,47,4.0
...,...,...,...
737,496,48,4.0
738,497,7,5.0
739,498,48,4.0
740,498,23,5.0


## Print statistics

Some statistics about the acquired data are shown.

In [6]:
# Statistics about data
arr_tourists = tourist_df["id"].unique()
arr_guides = guide_df["id"].unique()

n_tourists = len(arr_tourists)
n_guides = len(arr_guides)
n_interactions = len(rating_df)

print("Number of tourists: {:d}".format(n_tourists))
print("Number of guides: {:d}".format(n_guides))
print("Number of interactions: {:d}".format(n_interactions))

print("Average interaction per tourist: {:.2f}".format(n_interactions/n_tourists))
print("Average interaction per guide: {:.2f}".format(n_interactions/n_guides))
print("Sparsity: {:.2f} %".format((1-float(n_interactions)/(n_guides*n_tourists))*100))

Number of tourists: 500
Number of guides: 50
Number of interactions: 742
Average interaction per tourist: 1.48
Average interaction per guide: 14.84
Sparsity: 97.03 %


In [7]:
# Statistics about the distribution of ratings
print("Average rating: {:.6f}".format(rating_df.loc[:, 'rating'].mean()))
print("Maximum rating: {:.6f}".format(rating_df.loc[:, 'rating'].max()))
print("Minimum rating: {:.6f}".format(rating_df.loc[:, 'rating'].min()))

Average rating: 4.005391
Maximum rating: 6.000000
Minimum rating: 2.000000


## Create the URM

The **User Rating Matrix** describes the past interactions between tourists and guides, where rows represent tourists and columns represent guides. The values in the cells can be defined by an implicit or explicit approach:
- Explicit ratings are given directly by the tourists to the guides, according to a rating scale.
- Implicit ratings are obtained according to specific criteria based on tourists' behaviour, without asking for an opinion explicitly. The corresponding value of the interaction is set to 1 if we think that the guide could be interested in the guide, otherwise 0.

In our problem, we decide to use **explicit ratings** with a rating scale from 1 to 5.

In [8]:
# Create the User Rating Matrix in CSR format to facilitate the computation
# Row: tourist_id, Column: guide_id, Value: rating
URM_all = sps.csr_matrix(
    (rating_df["rating"].values,
    (rating_df["tourist_id"].values, rating_df["guide_id"].values))
)

URM_all

<500x50 sparse matrix of type '<class 'numpy.float64'>'
	with 742 stored elements in Compressed Sparse Row format>

In [9]:
# Define the portion of data used for training the model
# Since we are using fake data, we train the model on all the available data
URM_train = URM_all

## Create the ICM

The **Item Content Matrix** describes the guides with their own attributes. The rows indicates the guides, and the columns represent attributes. Each number in the ICM indicates how much important an attribute is in characterizing a guide.

In this case we started from the simplest form: the cell value is equal to 1 if the guide has that specific attribute, and 0 otherwise.

In [10]:
# Make a copy of the guide dataframe to build the ICM
icm_df = guide_df.copy(deep=True)

In [11]:
# Convert birthday information into categorical labels
# Classification criterion: if the guide is more than 40 years old or not.
def replace_birth_year(x):
    if x > 1984:
        return '20-40'
    else:
        return '40+'

# Apply the conversion to the dataframe
icm_df['birth_date'] = icm_df['birth_date'].apply(
    lambda x: replace_birth_year(pd.to_datetime(x, format="%Y-%m-%d").year)
)

In [12]:
# Convert number of years of experience into categorical labels
# Classification criteria:
# - a junior guide has less than 5 years of experience;
# - an experience guide has between 5 and 10 years of experience;
# - a senior guide has at least 10 years of experience.
def replace_experience(x):
    if x < 5:
        return 'junior'
    elif x < 10:
        return 'experienced'
    else:
        return 'senior'

# Apply conversion to the dataframe
icm_df['experience'] = icm_df['experience'].apply(replace_experience)

In [13]:
# Compute the mean value of prices as a reference for proper ranges
icm_df[['price']].mean(axis=0)

price    29.02
dtype: float64

In [14]:
# Split the price information into three ranges
# - cost < 25: low cost
# - 25 <= cost < 35: medium
# - cost >= 35: high

def replace_price(x):
    if x < 25:
        return 'low_cost'
    elif x < 35:
        return 'medium_cost'
    else:
        return 'high_cost'

# Apply conversion to the dataframe
icm_df['price'] = icm_df['price'].apply(replace_price)

In [15]:
# Show the dataframe
icm_df

,id,gender,name,birth_date,now_available,languages_spoken,price,education,biography,keywords,current_location,experience
0,0,male,Henry Robles,20-40,True,[english],medium_cost,high-school,Community development worker,[museums],"{'lat': 40.342693584880706, 'lon': 18.16438078...",experienced
1,1,female,Raven Burke,20-40,True,"[italian, dutch]",high_cost,middle-school,Quantity surveyor,"[cinema, rafting, history, wine]","{'lat': 40.3367551413547, 'lon': 18.1569120995...",junior
2,2,female,Nancy Keith,40+,True,"[chinese, french, english]",medium_cost,phd,Secretary/administrator,"[food, archeology, art]","{'lat': 40.362447049660204, 'lon': 18.14225129...",senior
3,3,male,Michael Mitchell,40+,True,[bulgarian],high_cost,bachelor,"Lighting technician, broadcasting/film/video","[countryside, rafting, art]","{'lat': 40.36584086550253, 'lon': 18.183002910...",senior
4,4,male,John Bailey,40+,True,"[deutsche, french]",medium_cost,master,Estate manager/land agent,"[countryside, tracking, beer]","{'lat': 40.354724889812935, 'lon': 18.20308322...",senior
5,5,male,Justin Allen,20-40,True,"[deutsche, dutch, bulgarian]",medium_cost,middle-school,Microbiologist,[],"{'lat': 40.35854623058413, 'lon': 18.183459879...",junior
6,6,male,Jose Russell,20-40,True,"[english, chinese, french]",medium_cost,middle-school,"Pharmacist, community","[rafting, art, sport]","{'lat': 40.35774331848761, 'lon': 18.163273185...",junior
7,7,male,Jack Smith,40+,True,"[spanish, dutch, french, italian]",medium_cost,bachelor,Theatre manager,"[museums, art, countryside, wine]","{'lat': 40.365193043282794, 'lon': 18.18548169...",experienced
8,8,male,Victor Gonzalez,20-40,True,"[bulgarian, english, chinese]",medium_cost,middle-school,Geochemist,"[literature, cinema, music]","{'lat': 40.35265552781134, 'lon': 18.151723396...",junior
9,9,male,John Brown,20-40,True,"[chinese, bulgarian, dutch, english]",high_cost,phd,"Development worker, international aid",[museums],"{'lat': 40.33774657696426, 'lon': 18.182064664...",senior


In [16]:
# Remove the columns that we would not consider as relevant static attributes for guides
icm_df.drop(labels=['name', 'now_available', 'current_location'], axis=1, inplace=True)

In [17]:
# Split the categorical attributes into separate columns
multiclass_attributes = ['gender', 'price', 'experience', 'birth_date', 'education', 'biography', 'languages_spoken', 'keywords']

# The result would have value 1 for each cell corresponding to an attribute owned by a guide
# and 0 otherwise
for n in multiclass_attributes:

    # Explode lists of attributes to a subset of columns
    s = icm_df[n].explode()

    # Augment the dataframe with columns corresponding to each category, which belongs to the set of attributes. The cell values are determined by the counts of occurrences for each element.
    # (if the intersection is not null, the cell has value 1, otherwise it is filled with 0)
    icm_df = icm_df.join(pd.crosstab(s.index, s).astype(object)).fillna(0)

    # Drop the original column
    icm_df.drop(labels=n,axis=1,inplace=True)

In [18]:
# Show the dataframe
icm_df

,id,female,male,high_cost,low_cost,medium_cost,experienced,junior,senior,20-40,...,countryside,food,history,literature,museums,music,rafting,sport,tracking,wine
0,0,0,1,0,0,1,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,1,1,0,1,0,0,0,1,0,1,...,0,0,1,0,0,0,1,0,0,1
2,2,1,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,3,0,1,1,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
4,4,0,1,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
5,5,0,1,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
6,6,0,1,0,0,1,0,1,0,1,...,0,0,0,0,0,0,1,1,0,0
7,7,0,1,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1
8,8,0,1,0,0,1,0,1,0,1,...,0,0,0,1,0,1,0,0,0,0
9,9,0,1,1,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0


In [19]:
# Print the list of attributes
attribute_list = icm_df.columns.tolist()
attribute_list

['id',
 'female',
 'male',
 'high_cost',
 'low_cost',
 'medium_cost',
 'experienced',
 'junior',
 'senior',
 '20-40',
 '40+',
 'bachelor',
 'high-school',
 'master',
 'middle-school',
 'phd',
 'Accountant, chartered public finance',
 'Analytical chemist',
 'Animal technologist',
 'Arboriculturist',
 'Armed forces operational officer',
 'Building services engineer',
 'Chartered public finance accountant',
 'Chief Technology Officer',
 'Child psychotherapist',
 'Clinical molecular geneticist',
 'Community arts worker',
 'Community development worker',
 'Conservator, furniture',
 'Copy',
 'Data scientist',
 'Designer, interior/spatial',
 'Development worker, international aid',
 'Educational psychologist',
 'Estate manager/land agent',
 'Field seismologist',
 'Financial adviser',
 'Financial manager',
 'Food technologist',
 'Geochemist',
 'Geophysicist/field seismologist',
 'Health and safety adviser',
 'Lecturer, higher education',
 'Lighting technician, broadcasting/film/video',
 'Magaz

In [20]:
# Convert the names of columns (attributes) into sequential numbers
def convert_index(x):
    # The first column contains the ids of the guide, keep its name
    if x == 'id':
        return x
    else:
        return attribute_list.index(x)

# Apply the transformation to the dataframe
icm_df.rename(mapper=convert_index, axis=1, inplace=True)
icm_df

,id,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,0,0,1,0,0,1,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,1,1,0,1,0,0,0,1,0,1,...,0,0,1,0,0,0,1,0,0,1
2,2,1,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,3,0,1,1,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
4,4,0,1,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
5,5,0,1,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
6,6,0,1,0,0,1,0,1,0,1,...,0,0,0,0,0,0,1,1,0,0
7,7,0,1,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1
8,8,0,1,0,0,1,0,1,0,1,...,0,0,0,1,0,1,0,0,0,0
9,9,0,1,1,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0


In [21]:
# Re-organize data structure for building the ICM

# The dataframe is modeled such that the column 'id' is identifier variable,
# and all other columns are "unpivoted" to the row axis.
# The result has only two non-identifier columns: 'label' (attribute) and 'value'.
icm_df = pd.melt(icm_df, id_vars='id', var_name='label')

# Filter the values: only the cells with value 1 are kept in the new dataframe
icm_df = icm_df[icm_df["value"]==1]
icm_df

,id,label,value
1,1,1,1
2,2,1,1
10,10,1,1
11,11,1,1
12,12,1,1
...,...,...,...
4149,49,83,1
4151,1,84,1
4157,7,84,1
4167,17,84,1


In [22]:
# Create the Item Content Matrix in CSR format to facilitate the computation
# Row: guide_id, Column: attribute_id, Value: 1 if the attribute is present for the guide
ICM_all = sps.csr_matrix(
    (icm_df["value"].values,
    (icm_df["id"].values, icm_df["label"].values))
)

ICM_all

<50x85 sparse matrix of type '<class 'numpy.int64'>'
	with 509 stored elements in Compressed Sparse Row format>

In [23]:
# Print the matrix in dense format
print(ICM_all.todense())

[[0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 1]
 [0 1 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 1 0]]


### Feature Engineering
It is possible to model the importance of the features by weighting them differently in the ICM: so we can attribute a higher value to the features that we consider more relevant for the recommendation problem, such as the languages spoken by the guides.

In the considered scenario, we assume that the **language** is a hard constraint with high priority in the matching process: for communication purpose, it is reasonable that the recommended guides and the tourist speak the same language. So we aim to first guarantee this constraint.

Other attributes, such as the experience or the educational background of the guides, can be further highlighted whenever necessary.

In [24]:
# Print the full list of attributes with their indices
for l in attribute_list:
    print(attribute_list.index(l), l)

0 id
1 female
2 male
3 high_cost
4 low_cost
5 medium_cost
6 experienced
7 junior
8 senior
9 20-40
10 40+
11 bachelor
12 high-school
13 master
14 middle-school
15 phd
16 Accountant, chartered public finance
17 Analytical chemist
18 Animal technologist
19 Arboriculturist
20 Armed forces operational officer
21 Building services engineer
22 Chartered public finance accountant
23 Chief Technology Officer
24 Child psychotherapist
25 Clinical molecular geneticist
26 Community arts worker
27 Community development worker
28 Conservator, furniture
29 Copy
30 Data scientist
31 Designer, interior/spatial
32 Development worker, international aid
33 Educational psychologist
34 Estate manager/land agent
35 Field seismologist
36 Financial adviser
37 Financial manager
38 Food technologist
39 Geochemist
40 Geophysicist/field seismologist
41 Health and safety adviser
42 Lecturer, higher education
43 Lighting technician, broadcasting/film/video
44 Magazine journalist
45 Maintenance engineer
46 Microbiolog

In [25]:
# Define which features we would like to model
# Some examples could be 'low_cost' to consider the cost of the guide,
# 'experienced' for the experience, '20-40' for the age, etc.
features_to_model = ['low_cost', 'experienced', '20-40']

# Define the importance of those features that we want to attribute in a positional way
# Default weight: 1
importance_weights = [1, 1, 1]

# Extract the columns representing languages, and attribute a higher weight
languages_columns = [61, 68]
languages_weight = 10

In [26]:
# Initialize the dictionaries for the chosen attributes and weights
feature_columns_dict = dict()
importance_weights_dict = dict()

# Add the language feature to the two dictionaries
feature_columns_dict.update({'language': languages_columns})
importance_weights_dict.update({'language': languages_weight})

In [27]:
# Build function to extract other features' indices and the corresponding weights to attribute
def extract_feature(name, i):
    # Update the dictionaries with indices and weights
    feature_columns_dict.update({name: attribute_list.index(name)})
    importance_weights_dict.update({name: importance_weights[i]})

# Execute the operation
for i in range(len(features_to_model)):
    extract_feature(features_to_model[i], i)

print(feature_columns_dict)
print(importance_weights_dict)

{'language': [61, 68], 'low_cost': 4, 'experienced': 6, '20-40': 9}
{'language': 10, 'low_cost': 1, 'experienced': 1, '20-40': 1}


In [28]:
# Create a copy of the original ICM to modify
new_icm_df = icm_df.copy(deep=True)

In [29]:
# Modify the cell values with respect to the weights we want to give
for feature in importance_weights_dict:

    # If the weight is not default, modify the values
    if importance_weights_dict[feature] > 1:

        # Trace all columns assigned to the languages
        if feature=='language':
            condition = (new_icm_df.label >= feature_columns_dict[feature][0]) & (new_icm_df.label <= feature_columns_dict[feature][1])

        # Trace the specific feature
        else:
            condition = (new_icm_df.label == feature_columns_dict[feature])

        # Find the rows with labels corresponding to the selected features, and update the values
        new_icm_df.loc[condition, 'value'] = importance_weights_dict[feature]

In [30]:
# Print the modified dataframe
new_icm_df

,id,label,value
1,1,1,1
2,2,1,1
10,10,1,1
11,11,1,1
12,12,1,1
...,...,...,...
4149,49,83,1
4151,1,84,1
4157,7,84,1
4167,17,84,1


In [31]:
# Build the modified ICM from the dataframe
# Like the URM, also the ICM is built in CSR format for computational purpose
# Row: guide_id, Column: attribute_id, Value: 1 if the attribute is present for the guide

ICM_modified = sps.csr_matrix(
    (new_icm_df["value"].values,
    (new_icm_df["id"].values, new_icm_df["label"].values))
)

ICM_modified

<50x85 sparse matrix of type '<class 'numpy.int64'>'
	with 509 stored elements in Compressed Sparse Row format>

In [32]:
# Define the portion of ICM used for training the model: here we use the entire ICM
ICM_train = ICM_modified

## Build the model

At this point all the necessary input data are prepared, and we proceed to build the recommendation algorithm.


### Similarity Function
A recommendation system learns from the past ratings and/or the attributes of the users. By evaluating the similarity between the guides, it is able to suggest guides that are similar to the ones that the tourist liked in the past, and also to recommend a guide liked by a tourist to similar tourists.

To determine the degree of similarity between the users, we choose to implement the **Cosine Similarity** function for the models. In our case, it consists in determining the number of common elements between two vectors $\vec{i}$ and $\vec{j}$ representing guides, and this can be computed with the **normalized dot product**:
$$    s_{ij}=\frac{\vec{i}\cdot \vec{j}}{|\vec{i}|_2\cdot |\vec{j}|_2}=cos(\theta)   .$$

For faster computation, we adopt a version of the cosine similarity based on vector products, with $M$ the reference matrix (URM or ICM):
$$ W_{i,I}
= cos(v_i, M_{I})
= \frac{v_i \cdot M_{I}}{|| v_i || IW_{I} + shrink}  ,$$
where
$$ IW_{i} = \sqrt{{\Sigma_{u \in U}{M_{u,i}^2}}}  .$$
The shrink term is introduced to take into account the support of the vectors: vectors with larger number of non-zero elements in them are statistically more significant.

Finally, the similarity values are stored in a **similarity matrix**, which establishes the pairwise correspondence between guides. This is fed to the models as weights.

In [33]:
# Define the cosine similarity function based on vector products
def vector_similarity(m: sps.csc_matrix, shrink: int):

    # Compute the Euclidean norm of each column of the matrix m
    item_weights = np.sqrt(
        np.sum(m.power(2), axis=0)
    ).A.flatten()

    # Find the number of items
    num_items = m.shape[1]

    # Get the transposed matrix
    matrix_t = m.T

    # Initialize the empty weight matrix
    weights = np.empty(shape=(num_items, num_items))

    # Compute the similarity values as mentioned in the previous formula
    # The results will be weights to give to the model
    for item_id in range(num_items):
        numerator = matrix_t.dot(m[:, item_id]).A.flatten()
        denominator = item_weights[item_id] * item_weights + shrink + 1e-6

        weights[item_id] = numerator / denominator

    # The elements on the diagonal of the similarity matrix are set to 0,
    # to force the fact that each element is not considered similar to itself!
    np.fill_diagonal(weights, 0.0)

    return weights

To solve our problem, we worked on two different approaches: **Collaborative Filtering** (CF) and **Content-Based Filtering** (CBF).

### Collaborative Filtering Model

**Collaborative Filtering** models rely on the opinions of a community of tourists, without the need of guides' attributes. It essentially focuses on identifying patterns and similarities in interactions between tourists and guides based on the tourists' feedback.

In this case, **item-based** collaborative filtering technique has been used. It calculates the similarity between each pair of guides, according to the number of tourists that have the same opinion on them.

In [34]:
# Define the item-based collaborative filtering recommender
class ItemCFRecommender(object):

    # Initialize the input matrix
    def __init__(self, URM):
        self.URM = URM
        
    # Fit the recommender with a chosen shrink value
    def fit(self, shrink=1):
        # Compute the pairwise similarity between guides using the URM
        self.W_matrix = vector_similarity(self.URM.tocsc(), shrink=shrink)
        # Print the weight matrix
        # with np.printoptions(threshold=np.inf):
        #     print(self.W_matrix)

    # Generate recommendations for each tourist
    # at: number of guides that we want to recommend to each tourist
    # exclude_seen: if we want to avoid recommending the guides that the tourist has rated before
    def recommend(self, user_id, at=None, exclude_seen=True):
        user_profile = self.URM[user_id]
        # Compute the scores using the dot product
        scores = user_profile.dot(self.W_matrix).ravel()

        # Filter the guides
        if exclude_seen:
            scores = self.filter_seen(user_id, scores)

        # Rank the guides
        ranking = scores.argsort()[::-1]
            
        return ranking[:at]
    
    # Guides that have been already rated by the tourist will be excluded in the recommendation
    def filter_seen(self, user_id, scores):
        # Retrieve the cells regarding the interactions of that specific tourist
        start_pos = self.URM.indptr[user_id]
        end_pos = self.URM.indptr[user_id+1]

        # Extract the user profile
        user_profile = self.URM.indices[start_pos:end_pos]

        # Set the score of already rated guides to the minimum value
        scores[user_profile] = -np.inf

        return scores

### Content-Based Filtering Model

A **Content-Based Filtering** model recommends guides to tourists based on the attributes of the guides themselves. It stands upon the assumption of recommending items similar to the ones a user liked in the past, and it requires a list of good quality attributes for the guides to work properly.

In this case **item-based** CBF technique has been used. It suggests guides to tourists according to the similarity weights computed on the guides: two guides are similar if they have a great part of attributes in common.

In [35]:
# Define the item-based content-based filtering recommender
class ItemCBFRecommender(object):

    # Initialize the input matrices
    def __init__(self, URM, ICM):
        self.URM = URM
        self.ICM = ICM

    # Fit the recommender with a chosen shrink value
    def fit(self, shrink=1):
        # Compute the pairwise similarity between guides using the transposed ICM
        self.W_matrix = vector_similarity(self.ICM.T.tocsc(), shrink=shrink)
        # with np.printoptions(threshold=np.inf):
        #     print(self.W_matrix)

    # Generate recommendations for each tourist
    # at: number of guides that we want to recommend to each tourist
    # exclude_seen: if we want to avoid recommending the guides that the tourist has rated before
    def recommend(self, user_id, at=None, exclude_seen=True):
        user_profile = self.URM[user_id]
        # Compute the scores using the dot product
        scores = user_profile.dot(self.W_matrix).ravel()

        # Filter the guides
        if exclude_seen:
            scores = self.filter_seen(user_id, scores)

        # Rank the guides
        ranking = scores.argsort()[::-1]

        return ranking[:at]

    # Guides that have been already rated by the tourist will be excluded in the recommendation
    def filter_seen(self, user_id, scores):

        # Retrieve the cells regarding the interactions of that specific tourist
        start_pos = self.URM.indptr[user_id]
        end_pos = self.URM.indptr[user_id+1]

        # Extract the user profile
        user_profile = self.URM.indices[start_pos:end_pos]

        # Set the score of already rated guides to the minimum value
        scores[user_profile] = -np.inf

        return scores

## Fit the model
The recommenders are built and fit on the available input data.

In [36]:
# Select if building a collaborative filtering model or a content-based filtering model
# Available options: 'cf', 'cbf'
model_type = 'cbf'

In [37]:
# Collaborative filtering model
# Input data: URM
# The shrink value is a non-negative number that can be tuned
if model_type == 'cf':
    recommender = ItemCFRecommender(URM_train)
    recommender.fit(shrink=0.5)

In [38]:
# Content-based filtering model
# Input data: URM and ICM
# The shrink value is a non-negative number that can be tuned
if model_type == 'cbf':
    recommender = ItemCBFRecommender(URM_train, ICM_train)
    recommender.fit(shrink=0.5)

## Generate outputs

Here the generation of recommendations of guides to each tourist is reported.

In [39]:
# Set the number of guides to recommend to each tourist
n_recommendations_per_tourist = 5

In [40]:
# Generate recommendations for each tourist
recommendations = []

for i,id in tqdm(enumerate(arr_tourists)):
    # at: number of guides to recommend to each tourist
    # exclude_seen: if we want to exclude the guides already rated by this tourist
    rec = recommender.recommend(id, at=n_recommendations_per_tourist, exclude_seen=True)
    rec_list = rec

    # Produce a string with the ids of the recommended guides
    rec_row = ' '.join(str(s) for s in rec_list)
    recommendations.append(rec_row)

500it [00:00, 17857.07it/s]


In [41]:
# Print recommendations for the first 10 users
for i in range(10):
    print("User " + str(arr_tourists[i]) + " - recommended guides: " + recommendations[i])

User 0 - recommended guides: 28 38 40 27 22
User 1 - recommended guides: 14 12 7 2 6
User 2 - recommended guides: 5 24 10 19 42
User 3 - recommended guides: 34 29 21 44 14
User 4 - recommended guides: 4 31 30 43 33
User 5 - recommended guides: 23 10 5 30 8
User 6 - recommended guides: 29 44 8 48 9
User 7 - recommended guides: 29 44 8 48 9
User 8 - recommended guides: 20 4 6 2 19
User 9 - recommended guides: 48 3 42 10 23


In [42]:
# Show the output dataframe
result_df = pd.DataFrame(
    data = {'tourist_id': arr_tourists,
            'guides': recommendations}
)

result_df

,tourist_id,guides
0,0,28 38 40 27 22
1,1,14 12 7 2 6
2,2,5 24 10 19 42
3,3,34 29 21 44 14
4,4,4 31 30 43 33
...,...,...
495,495,30 4 20 33 6
496,496,42 11 3 5 9
497,497,12 45 17 18 37
498,498,42 11 3 19 8


Let's randomly choose a tourist and check for information of recommended guides.

In [43]:
# Show some examples: select a tourist by id to visualize the received recommendations
sample_tourist = 20

# Show the selected tourist
pd.DataFrame(tourist_df.loc[sample_tourist,:])

,20
id,20
languages,['bulgarian']
keywords,"['food', 'countryside', 'archeology', 'literat..."


In [44]:
# Extract the indices of the recommended guides
sample_guide_list = list(map(int, recommendations[sample_tourist].split(" ")))

# Show the list of recommended guides
guide_df.loc[sample_guide_list,:]

,id,gender,name,birth_date,now_available,languages_spoken,price,education,biography,keywords,current_location,experience
16,16,male,Benjamin Taylor,1959-06-11,True,"[spanish, bulgarian, french]",32,middle-school,"Designer, interior/spatial","[countryside, music, beer]","{'lat': 40.32347428820915, 'lon': 18.164571467...",20
32,32,female,Sarah Cortez,1960-11-16,True,"[french, bulgarian]",28,phd,Chartered public finance accountant,"[archeology, music]","{'lat': 40.343414579847, 'lon': 18.18565562805...",41
48,48,female,Jessica Mitchell,1971-11-24,True,"[italian, bulgarian]",30,high-school,Geophysicist/field seismologist,[],"{'lat': 40.34687214441545, 'lon': 18.186650378...",28
42,42,female,Melinda Mccoy,2003-03-17,True,[bulgarian],34,bachelor,Animal technologist,"[tracking, sport]","{'lat': 40.349356335154205, 'lon': 18.18610449...",2
3,3,male,Michael Mitchell,1981-08-22,True,[bulgarian],46,bachelor,"Lighting technician, broadcasting/film/video","[countryside, rafting, art]","{'lat': 40.36584086550253, 'lon': 18.183002910...",18
